In [36]:
import numpy
import matplotlib
import pandas
import os
import math
import hashlib

# Task 1
## Rotate Left Function 

### Explanation:
- The bits go past the left and around to the right.
- This creates a circle shift.
- 4-bit (`0b1111`) to keep results within 4 bits.

## Rotate Right Function
- This is just a reverse of rotate left.
- Quite useful in cryptographic operations that require predictable bit-level editing.

## Choice Function
- Used to choose bits from 'y' or 'z' based on the value of 'x'.
- If a bit in 'x' is 1, take the bit from 'y', if it's 0, take it from 'z'.
- Used in SHA-256 and other hash functions.

## Majority Function
- Looks at each bit and picks the value that appears in at least two of the three inputs.
- Similar to a "voting system" for each bit - if two or more inputs agree, thats the winning value.
- It is used in hash functions to mix bits and make the output harder to predict.

In [37]:
def rotl(x, n=1):
    # Rotate bits of a 32-bit unsigned integer to the left by n positions
    n = n % 4  
    return ((x << n) & 0b1111) | (x >> (4 - n))  

def rotr(x, n=1):
    # Rotate bits of a 4-bit unsigned integer to the right by n positions
    n = n % 4  
    return ((x >> n) | (x << (4 - n))) & 0b1111   

def ch(x, y, z):
    # Choose bits from y where x is 1, and from z where x is 0
    return (x & y) ^ (~x & z)

def maj(x, y, z):
     # Choose 1 if at least two of x, y, or z are 1
    return (x & y) | (x & z) | (y & z)

In [38]:
x, y, z = 0b1100, 0b1010, 0b0110
rotated_x = rotl(x, 1)  # Rotate left by 1 bit

print(f"Original: {bin(x)} ({x})")
print(f"x = {bin(x)} ({x})")
print(f"y = {bin(y)} ({y})")
print(f"z = {bin(z)} ({z})")

# Test rotate left
print(f"Rotated Left: {bin(rotated_x)} ({rotated_x})")

# Test rotate right
rotr_x = rotr(x, 1)
print(f"Rotated Right: {bin(rotr_x)} ({rotr_x})")

# Test ch function
ch_result = ch(x, y, z)
print(f"ch(x, y, z) = {bin(ch_result)} ({ch_result})")

# Test maj function
maj_result = maj(x, y, z)
print(f"maj(x, y, z) = {bin(maj_result)} ({maj_result})")

Original: 0b1100 (12)
x = 0b1100 (12)
y = 0b1010 (10)
z = 0b110 (6)
Rotated Left: 0b1001 (9)
Rotated Right: 0b110 (6)
ch(x, y, z) = 0b1010 (10)
maj(x, y, z) = 0b1110 (14)


# END TO TASK 1
## Task 2: Conversion, C - Python

### Explanation:
- This task requires me to convert a basic C-style hash function to Python.
- It loops over each character in a string and builds a running hash value.
- 'ord(char)' gets the Unicode value of each character.
- The result is taken modulo 101 to limit it to a fixed range.

### Test Case:
- I tested the function on a set of strings.
- The outputs match expectations and show how different inputs map to different hash values.

In [39]:
def hash_function(s):
    hashval = 0
    for char in s:
        hashval = ord(char) + 31 * hashval  
    return hashval % 101

In [40]:
# Test cases with expected outputs
test_strings = ["hello", "world", "hash", "function", "test", "python"]

for s in test_strings:
    print(f"hash_function('{s}') = {hash_function(s)}")

hash_function('hello') = 17
hash_function('world') = 34
hash_function('hash') = 15
hash_function('function') = 100
hash_function('test') = 86
hash_function('python') = 91


# END TO TASK 2
## Task 3: SHA256

### Explanation:
- This task is to show how SHA-256 prepares messages before hashing.
- The input file is read as bytes
- Padding starts with a '1' bit, followed by enough '0' bits to make the message length 64 bits short of a 
  multiple of 512.
- Then the length of the original input is added as a 64-bit big-endian integer.
- This makes sure the message is the correct size for SHA-256's algorithm to process.

In [41]:
def sha256_pad(file_path):
    with open(file_path, 'rb') as f:
        original = f.read()

    length = len(original) * 8  # length in bits
    data = original + b'\x80'

    while (len(data) * 8) % 512 != 448:
        data += b'\x00'

    data += length.to_bytes(8, 'big')

    padding = data[len(original):]
    print(' '.join(f'{b:02x}' for b in padding))

In [42]:
# Create the test file with some content
with open("test.txt", "w") as f:
    f.write("abc")

# Then run the padding function
sha256_pad("test.txt")

80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


# END TO TASK 3
## Task 4: Prime Numbers

### Explanation: Calculate the first 100 prime numbers using two different algorithms:
### Method 1: Trial Division
This method will check if a number is divisible by any number from 2 - √n
If it is not divisible, it's considered prime. This method is slower but simpler

### Method 2: Sieve of Eratosthenes
This method creates a list of possible primes and removes multiples of each number.
More efficient and better for generating many primes.

In [43]:
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
        return True

In [44]:
def primes_trial_division(limit):
    primes = []
    num = 2
    while len(primes) < limit:
        if is_prime(num):
            primes.append(num)
        num += 1
    return primes

In [45]:
def sieve_of_eratosthenes(n):
    size = 600
    is_prime = [True] * size
    is_prime[0] = is_prime[1] = False

    for i in range(2, int(size**0.5) + 1):
        if is_prime[i]:
            for j in range(i*i, size, i):
                is_prime[j] = False
            
    primes = [i for i, val in enumerate(is_prime) if val]
    return primes[:n]

In [46]:
print("Trial Division:\n", primes_trial_division(100))
print("\nSieve of Eratosthenes:\n", sieve_of_eratosthenes(100))

Trial Division:
 [5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203]

Sieve of Eratosthenes:
 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


### Comparing the Results:
Both methods do the job, they correctly generate the first 100 prime numbers.
But Sieve of Eratosthenes is faster for bigger listrs, but Trial Division is much easier to understand.

# END TO TASK 4

## Task 5: Roots

### Explanation:
- This task focuses on isolating the fractional portion of each square root and expressing it as a 32-bit integer.
- First step is to take the square root of each prime.
- Then we only keep the decimal part.
- Then that is multiplied by 2^32 and converted to an interger to get a 32-bit value.

In [47]:
def get_root_bits(primes):
    results = []
    for p in primes:
        root = math.sqrt(p)
        frac = root % 1
        bits = int (frac * (2**32))
        results.append(bits)
    return results

In [48]:
primes = primes_trial_division(100)
root_bits = get_root_bits(primes)

for i, bits in enumerate(root_bits):
     print(f"Prime: {primes[i]} → 32-bit frac: {bits:#010x}")

Prime: 5 → 32-bit frac: 0x3c6ef372
Prime: 7 → 32-bit frac: 0xa54ff53a
Prime: 9 → 32-bit frac: 0x00000000
Prime: 11 → 32-bit frac: 0x510e527f
Prime: 13 → 32-bit frac: 0x9b05688c
Prime: 15 → 32-bit frac: 0xdf7bd629
Prime: 17 → 32-bit frac: 0x1f83d9ab
Prime: 19 → 32-bit frac: 0x5be0cd19
Prime: 21 → 32-bit frac: 0x9523ae45
Prime: 23 → 32-bit frac: 0xcbbb9d5d
Prime: 25 → 32-bit frac: 0x00000000
Prime: 27 → 32-bit frac: 0x32370b90
Prime: 29 → 32-bit frac: 0x629a292a
Prime: 31 → 32-bit frac: 0x9159015a
Prime: 33 → 32-bit frac: 0xbe9ba858
Prime: 35 → 32-bit frac: 0xea843464
Prime: 37 → 32-bit frac: 0x152fecd8
Prime: 39 → 32-bit frac: 0x3eb83056
Prime: 41 → 32-bit frac: 0x67332667
Prime: 43 → 32-bit frac: 0x8eb44a87
Prime: 45 → 32-bit frac: 0xb54cda58
Prime: 47 → 32-bit frac: 0xdb0c2e0d
Prime: 49 → 32-bit frac: 0x00000000
Prime: 51 → 32-bit frac: 0x2434a74b
Prime: 53 → 32-bit frac: 0x47b5481d
Prime: 55 → 32-bit frac: 0x6a8bfbea
Prime: 57 → 32-bit frac: 0x8cc1f315
Prime: 59 → 32-bit frac: 0xae5f

### Research Perspective: 
- This technique is used when designing ryptographic algorithms like **SHA-256** 
- These values look random, but they always come out the same — which is perfect for something that needs to be both secure and reliable.
# END TO TASK 5

## Task 6: Proof of Work

## Setup: 
- Use this line to downloade the requirements to do the task - !pip install nltk
- Run this import nltk nltk.download('words') 

## Explanation:
- This task finds the word in the English language that has the most `0` bits at the **start of its SHA-256 
hash**.
- SHA-256 is a secure hash function used in cryptography and blockchain.
- This idea is part of the “proof of work” system used in things like Bitcoin mining.


In [49]:
def leading_zeros(word):
    hash_hex = hashlib.sha256(word.encode()).hexdigest()
    hash_bin = bin(int(hash_hex, 16))[2:].zfill(256)
    return len(hash_bin) - len(hash_bin.lstrip('0'))  

In [50]:
with open("words.txt") as f:
    word_list = [line.strip().lower() for line in f if line.strip().isalpha()]
    
# Track the best result
best_word = ""
max_zeros = 0

for word in word_list:
    zeros = leading_zeros(word)
    if zeros > max_zeros:
        best_word = word
        max_zeros = zeros

In [51]:
print(f"Best word: '{best_word}' with {max_zeros} leading zero bits in SHA-256 hash.")

Best word: 'grady' with 15 leading zero bits in SHA-256 hash.


# END TO TASK 6

## Task 7: Turing Machines

### Research Perspective: 


# END TO TASK 7